<a href="https://colab.research.google.com/github/soltiste/ii-in-economi/blob/main/Lab7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Загрузите объекты из новостного датасета 20 newsgroups, относящиеся к категориям "космос"и "атеизм"(инструкция приведена выше).
2. Вычислите TF-IDF-признаки для всех текстов. Обратите внимание, что в этом задании мы предлагаем вам вычислить TF-IDF по
всем данным. При таком подходе получается, что признаки на обучающем множестве используют информацию из тестовой выборки — но такая ситуация вполне законна, поскольку мы не используем значения целевой переменной из теста. На практике нередко
встречаются ситуации, когда признаки объектов тестовой выборки
известны на момент обучения, и поэтому можно ими пользоваться
при обучении алгоритма.
3. Подберите минимальный лучший параметр C из множества [10−5
, 10−4
, ...10^4
, 10^5
]
для SVM с линейным ядром (kernel=’linear’) при помощи кроссвалидации по 5 блокам. Укажите параметр random_state=241 и
для SVM, и для KFold. В качестве меры качества используйте долю верных ответов (accuracy).
4. Обучите SVM по всей выборке с лучшим параметром C, найденным
на предыдущем шаге.
5. Найдите 10 слов с наибольшим по модулю весом. Они являются
ответом на это задание. Укажите их через запятую, в нижнем регистре, в лексикографическом порядке.

In [27]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import KFold, GridSearchCV
from sklearn.svm import SVC

In [28]:
newsgroups = datasets.fetch_20newsgroups(subset='all',
                                         categories=['alt.atheism',
                                                     'sci.space'])
X = newsgroups.data
y = newsgroups.target

In [29]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(X)

In [30]:
grid = {'C': np.power(10.0, np.arange(-5, 6))}
cv = KFold(n_splits=5, shuffle=True, random_state=241)
clf = SVC(kernel='linear', random_state=241)
gs = GridSearchCV(clf, grid, scoring='accuracy', cv=cv)
gs.fit(tfidf_matrix, y)
C = gs.best_params_.get('C')
C

1.0

In [31]:
clf = SVC(C = C, kernel='linear', random_state=241)
clf.fit(tfidf_matrix, y)

SVC(kernel='linear', random_state=241)

In [45]:
words = np.array(tfidf_vectorizer.get_feature_names_out())
data_abs = abs(clf.coef_.data)
word_matrix = pd.DataFrame(data_abs,
                           index=words[clf.coef_.indices],
                           columns=["weight"])

In [46]:
word_matrix_sorted = word_matrix.sort_values('weight', ascending=False)
word_matrix_sorted.head(10)

,weight
space,2.663165
god,1.920379
atheism,1.254690
atheists,1.249180
moon,1.201611
sky,1.180132
religion,1.139081
bible,1.130612
keith,1.097094
sci,1.029307
